# Serialization demo

In [1]:
import json
import numpy as np
import pickle

## pickle is simple but can be dangerous

In [2]:
class SomethingSimple:
    def __init__(self, foo, bar):
        self.foo = foo
        self.bar = bar


In [3]:
simple = SomethingSimple(foo=3, bar='two')

with open('simple.pickle', 'wb') as f:
    pickle.dump(simple, f)

with open('simple.pickle', 'rb') as f:
    simple_bis = pickle.load(f)

type(simple_bis), simple_bis.foo, simple_bis.bar

(__main__.SomethingSimple, 3, 'two')

In [4]:
class SomethingSimple:
    def __init__(self, foo, bla):
        self.foo = foo
        self.bla = bla

In [5]:
with open('simple.pickle', 'rb') as f:
    simple_bis = pickle.load(f)

simple_bis.__dict__

{'foo': 3, 'bar': 'two'}

Even worse when you have a simple class name change

In [6]:
# Simulate a name change of the SomethingSimple class
del SomethingSimple

class Simple:
    def __init__(self, foo, bar):
        self.foo = foo
        self.bar = bar

with open('simple.pickle', 'rb') as f:
    simple_bis = pickle.load(f)

simple_bis.__dict__

AttributeError: Can't get attribute 'SomethingSimple' on <module '__main__'>

## JSON is still quite simple, and allows you a closer control

In [7]:
class SomethingNice:

    def __init__(self, foo, bar):
        self.foo = foo
        self.bar = bar

    @classmethod
    def from_json(cls, fname):
        with open(fname, 'r') as f:
            dump = json.load(f)
        return cls(**dump)

    def to_json(self, fname):
        with open(fname, 'w') as f:
            json.dump(self.__dict__, f)


In [8]:
so_nice = SomethingNice(foo=3, bar='two')
so_nice.__dict__


{'foo': 3, 'bar': 'two'}

In [9]:
so_nice.to_json('nice.json')
!cat ./nice.json

{"foo": 3, "bar": "two"}

In [10]:
so_nice_again = SomethingNice.from_json('nice.json')
so_nice_again.__dict__

{'foo': 3, 'bar': 'two'}

In [11]:
not_so_nice = SomethingNice(foo=3, bar=np.array([1.2, 3.4]))
not_so_nice.__dict__

{'foo': 3, 'bar': array([1.2, 3.4])}

In [12]:
not_so_nice.to_json('not_so_nice.json')

TypeError: Object of type 'ndarray' is not JSON serializable

In [13]:
class SomethingWorking:

    def __init__(self, foo, data):
        self.foo = foo
        self.data = data

    @classmethod
    def from_json(cls, fname):
        with open(fname, 'r') as f:
            dump = json.load(f)
        dump['data'] = np.array(dump['data'])
        return cls(**dump)

    def to_json(self, fname):
        dump = {
            'foo': self.foo,
            'data': self.data.tolist(),
        }
        with open(fname, 'w') as f:
            json.dump(dump, f)


In [14]:
not_so_nice = SomethingWorking(foo=3, data=np.array([[1, 2], [3,4 ]]))
not_so_nice.to_json('not_so_nice.json')
!cat not_so_nice.json


{"foo": 3, "data": [[1, 2], [3, 4]]}